In [10]:
from pathlib import Path
import nibabel as nib
import numpy as np
from tqdm import tqdm

root_path = Path("/mnt/materials/SIRF/MathPlusBerlin/DATA/ACDC-Daten/")

fpath_output = Path("/home/jovyan/InputData/")
fpath_output.mkdir(exist_ok=True, parents=True)


In [11]:
def get_subdirs(path: Path):
    list_paths = []
    for path in Path(path).iterdir():
        if path.is_dir():
            list_paths.append(path)
    
    return list_paths

dirs_acdc = get_subdirs(root_path)


In [12]:
def crop_image(imgarr, cropped_size):
    
    imgdims = np.array(imgarr.shape)//2
    xstart = imgdims[0] - cropped_size//2
    xend = imgdims[0] + cropped_size//2
    
    ystart = imgdims[1] - cropped_size//2
    yend = imgdims[1] + cropped_size//2
    

    return imgarr[xstart:xend, ystart:yend,...]

def get_cropped_image(path: Path):
    
    img = nib.load(str(path / "image.nii.gz"))

    cropsize=128
    cropped_arr = crop_image(img.get_fdata(), cropsize)

    return np.squeeze(cropped_arr)

def get_formatted_image(path: Path, num_phases, time_usf):
    img = get_cropped_image(path)

    if img.shape[3] < min_num_phases:
        return None

    img = img[...,1:min_num_phases:time_usf]
    
    return img



In [15]:
min_num_phases = 24
time_usf = 2

i=0
for curr_dir in tqdm(dirs_acdc):
    list_patients = get_subdirs(curr_dir)
    
    for pat in list_patients:
        
        img = get_formatted_image(pat, min_num_phases, time_usf)
        if img is None:
            break
            
        for islice in range(img.shape[2]):
            dat = np.squeeze(img[:,:,islice,:])
            
            dat = np.swapaxes(dat, 0, 2)
            nii = nib.Nifti1Image(dat, np.eye(4))
            
            nib.save(nii, str(fpath_output / "img_{}.nii".format(i)))
            i += 1
        

100%|██████████| 4/4 [00:10<00:00,  2.63s/it]


Now we have to prepare the k-space data

In [16]:
import sirf.Gadgetron as pMR
import numpy as np


fname_template = fpath_output / "template/cine_128_30ph_bc.h5"
template_data = pMR.AcquisitionData(str(fname_template))

template_data = pMR.preprocess_acquisition_data(template_data)
template_phase = template_data.get_ISMRMRD_info("phase")

index_all_acquis = np.arange(template_data.number())

index_interesting_acquis = template_phase < (min_num_phases// time_usf)
index_interesting_acquis = index_all_acquis[index_interesting_acquis]
template_data = template_data.get_subset(index_interesting_acquis)

template_data = aux.undersample_cartesian_data(template_data)

Started reading acquisitions from /home/jovyan/InputData/template/cine_128_30ph_bc.h5
0%..10%..20%..30%..40%..50%..60%..70%..80%..90%..100%..
Finished reading acquisitions from /home/jovyan/InputData/template/cine_128_30ph_bc.h5


NameError: name 'aux' is not defined

In [ ]:
import sirf.Reg as pReg


csm = pMR.CoilSensitivityData()
csm.calculate(template_data)
csm.fill(1 + 0. * 1j)

template_img = pMR.ImageData()
template_img.from_acquisition_data(template_data)

E = pMR.AcquisitionModel(acqs=template_data, imgs=template_img)
E.set_coil_sensitivity_maps(csm)

path_input = fpath_output

list_stacks = sorted(path_input.glob("img*"))

template_fwd = pMR.ImageData()
template_fwd.from_acquisition_data(template_data)


for f in tqdm(list_stacks):
        
    fileidx = str(f.stem).split('_')[1]
        
    nii = pReg.ImageData(str(f))
    template_fwd.fill(nii.as_array())
    
    rd = E.forward(template_fwd)
    
    fout = fpath_output / "y_{}.h5".format(fileidx)
    rd.write(str(fout))
    

  6%|▌         | 28/488 [00:12<03:25,  2.24it/s]


Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

 12%|█▏        | 57/488 [00:24<02:58,  2.41it/s]

TI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NI

 17%|█▋        | 85/488 [00:36<02:50,  2.37it/s]

uld cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, c

 23%|██▎       | 114/488 [00:48<02:46,  2.25it/s]

_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYPE_FLOAT32.

Decreasing number of bytes per pixel, could cause loss of accuracy.
Input data type was NIFTI_TYPE_FLOAT64, converting to NIFTI_TYP

 28%|██▊       | 139/488 [01:00<02:34,  2.27it/s]